In [1]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import platform
plt.style.use('seaborn')
from datetime import datetime
import json

from metric import E1_loss, E2_loss, total_loss
from models import classifier, cnn_model, conv_block, cnn_parallel
from utils import train_model, eval_model, dfDataset, weights_init

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from sklearn.model_selection import KFold, train_test_split
from torchsummary import summary

### class, function...

In [2]:
class dfDataset(Dataset):
    def __init__(self, x, y):
        self.data = x
        self.target = y
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return self.data[index], self.target[index]
    
def weights_init(m, initializer = nn.init.kaiming_uniform_):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        initializer(m.weight)
        
def train_model(model, train_data, weight, optimizer, loss_func):
    loss_sum = 0
    for i, (x, y) in enumerate(train_data):
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()
        pred = model(x)
        loss = loss_func(pred, y)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    
    return loss_sum / len(train_data)

def eval_model(model, val_data, loss_func):
    with torch.no_grad():
        loss = 0
        for i, (x, y) in enumerate(val_data):
            x = x.cuda()
            y = y.cuda()
            pred = model(x)
            loss += loss_func(pred, y).item()
    return loss / len(val_data)

class conv_bn(nn.Module):
    def __init__(self, i_f, o_f, fs):
        super(conv_bn, self).__init__()
        self.conv = nn.Conv2d(i_f, o_f, fs, stride = (2, 1))
        self.act = nn.ELU()
        self.bn = nn.BatchNorm2d(o_f)
        #self.pool = nn.MaxPool2d(kernel_size=(2, 1), stride= (2, 1))
    def forward(self, x):
        x = self.bn(self.act(self.conv(x)))
        #return self.pool(x)
        return x
    
class conv_block(nn.Module):
    def __init__(self, h_list, input_shape, fs):
        '''
        input_shape : not include batch_size
        '''
        
        super(conv_block, self).__init__()
        self.input_shape = input_shape
        self.fs = fs
        convs = []
        for i in range(len(h_list)):
            if i == 0:
                convs.append(conv_bn(self.input_shape[0], h_list[i], fs))
            else:
                convs.append(conv_bn(h_list[i-1], h_list[i], fs))
        self.convs = nn.Sequential(*convs)
    
    def forward(self, x):
        return self.convs(x)
    
class classifier(nn.Module):
    def __init__(self, h_list, input_size, output_size):
        super(classifier, self).__init__()
        layers = []
        for i in range(len(h_list)):
            if i == 0:
                layers.append(nn.Linear(input_size, h_list[0]))
            else:
                layers.append(nn.Linear(h_list[i-1], h_list[i]))
            layers.append(nn.ELU())
            
        layers.append(nn.Linear(h_list[i], output_size))
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)
    
class cnn_model(nn.Module):
    def __init__(self, cnn_block, fc_block):
        super(cnn_model, self).__init__()
        self.cnn = cnn_block
        self.fc = fc_block
    def forward(self, x):
        x = self.cnn(x)
        x = x.flatten(start_dim = 1)
        return self.fc(x)

def E1_loss(y_pred, y_true):
    _t, _p = y_true, y_pred
    
    return torch.mean(torch.mean((_t - _p) ** 2, axis = 1)) / 2e+04

def E2_loss(y_pred, y_true):
    _t, _p = y_true, y_pred
    
    return torch.mean(torch.mean((_t - _p) ** 2 / (_t + 1e-06), axis = 1))

- sequential predict(XY-> V-> M)
- augmentation(noise add)
- channel concat

### Configuration

In [3]:
EPOCH = 100
base_lr = 0.001
now = datetime.strftime(datetime.now(), '%Y%m%d-%H%M%S')
save_path = './model/{}'.format(now)
initialize = True
print_summary = True
batch_size = 256
nfold = 10

### load dataset

In [4]:
if platform.system() == 'Windows':
    root_dir = 'D:/datasets/KAERI_dataset/'
else:
    root_dir = '/home/bskim/project/kaeri/KAERI_dataset/'

train_f = pd.read_csv(os.path.join(root_dir, 'train_features.csv'))
train_t = pd.read_csv(os.path.join(root_dir, 'train_target.csv'))
test_f = pd.read_csv(os.path.join(root_dir, 'test_features.csv'))

train_f = train_f[['Time','S1','S2','S3','S4']].values
train_f = train_f.reshape((-1, 1, 375, 5))#.astype(np.float32)

test_f = test_f[['Time','S1','S2','S3','S4']].values
test_f = test_f.reshape((-1, 1, 375, 5))#.astype(np.float32)
test_f = torch.FloatTensor(test_f)

### Train

In [5]:
def kfold_train(name, feature, target):
    print('{} train...'.format(name))

# make dataset
    train_target = target[list(name)].values#.astype(np.float32)

    # trainx, valx, trainy, valy = train_test_split(train_f, train_target, test_size = 0.2, shuffle = True, random_state = 38)
    fold = KFold(nfold, shuffle = True, random_state= 25)
    loss_per_cv = []
    for i, (train_idx, val_idx) in enumerate(fold.split(feature, y = train_target)):
        print('fold {}'.format(i+1))
        trainx = feature[train_idx]
        valx = feature[val_idx]
        trainy = train_target[train_idx]
        valy = train_target[val_idx]

        train_dataset = dfDataset(trainx.astype(np.float32), trainy)
        train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
        val_dataset = dfDataset(valx.astype(np.float32), valy)
        val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)

        n_features = feature.shape[-1]
        #fc = classifier([128, 64, 32, 16], input_size = 256*2*5, output_size = len(name))
        #conv = conv_block([32, 64, 128, 256, 256, 256, 256], [1, 375, 5], (3, 1))
        conv = conv_block([16, 32, 64, 128, 256, 256, 256], [1, 375, n_features], (3, 1))
        fc = classifier([128, 64, 32, 16], input_size = 256*1*n_features, output_size = len(name))
        # define model
        model = cnn_model(conv, fc)
        optimizer = torch.optim.Adam(model.parameters(), lr = base_lr)

        if name == 'XY':
            criterion = E1_loss
        else:
            criterion = E2_loss

        model = model.cuda()
        if initialize:
            model.apply(weights_init)

        curr_loss = 1e+7
        os.makedirs(save_path) if not os.path.exists(save_path) else None
        #train
        for ep in range(1, EPOCH + 1):
            model.train()
            loss = train_model(model, train_loader, criterion, optimizer, criterion)
            model.eval()
            val_loss =eval_model(model, val_loader, criterion)
            if curr_loss > val_loss:
                print('[{}] : train loss {:4f}, val loss drop {:.4f} to {:.4f}'.format(ep, np.mean(loss), curr_loss, val_loss))
                curr_loss = val_loss
                torch.save(model.state_dict(), os.path.join(save_path, 'model_{}_fold{}.pt'.format(name, i+1)))
        loss_per_cv.append(curr_loss)
    return loss_per_cv           

In [8]:
# train XY
loss = kfold_train('XY',train_f, train_t)

XY train...
fold 1
[1] : train loss 2.947474, val loss drop 10000000.0000 to 2.2058
[2] : train loss 1.811406, val loss drop 2.2058 to 0.9758
[3] : train loss 0.585628, val loss drop 0.9758 to 0.5168
[5] : train loss 0.036457, val loss drop 0.5168 to 0.0501
[7] : train loss 0.009243, val loss drop 0.0501 to 0.0085
[10] : train loss 0.005460, val loss drop 0.0085 to 0.0052
[11] : train loss 0.003609, val loss drop 0.0052 to 0.0031
[18] : train loss 0.002623, val loss drop 0.0031 to 0.0029
[20] : train loss 0.001639, val loss drop 0.0029 to 0.0013
[49] : train loss 0.001369, val loss drop 0.0013 to 0.0010
[54] : train loss 0.000932, val loss drop 0.0010 to 0.0007
[69] : train loss 0.001058, val loss drop 0.0007 to 0.0007
[72] : train loss 0.001392, val loss drop 0.0007 to 0.0006
[83] : train loss 0.000539, val loss drop 0.0006 to 0.0004
fold 2
[1] : train loss 3.054003, val loss drop 10000000.0000 to 2.3390
[2] : train loss 2.062557, val loss drop 2.3390 to 1.0744
[3] : train loss 0.6704

In [35]:
add_feature = train_t[['X','Y']].values.reshape((2800,1 , 1, 2))
add_feature = np.repeat(add_feature, 375, axis = 2)
trainX = np.concatenate((train_f, add_feature), axis = -1)

In [ ]:
# train V using XY
loss = kfold_train('V',trainX, train_t)

V train...
fold 1
[1] : train loss 4.264157, val loss drop 10000000.0000 to 0.5841
[2] : train loss 0.133741, val loss drop 0.5841 to 0.0779
[3] : train loss 0.052556, val loss drop 0.0779 to 0.0429
[4] : train loss 0.028958, val loss drop 0.0429 to 0.0267
[5] : train loss 0.018431, val loss drop 0.0267 to 0.0230
[6] : train loss 0.013264, val loss drop 0.0230 to 0.0138
[9] : train loss 0.010436, val loss drop 0.0138 to 0.0102
[10] : train loss 0.008032, val loss drop 0.0102 to 0.0095
[13] : train loss 0.007572, val loss drop 0.0095 to 0.0085
[16] : train loss 0.004096, val loss drop 0.0085 to 0.0051
[21] : train loss 0.003869, val loss drop 0.0051 to 0.0036
[36] : train loss 0.003180, val loss drop 0.0036 to 0.0031
[41] : train loss 0.001347, val loss drop 0.0031 to 0.0031
[42] : train loss 0.001513, val loss drop 0.0031 to 0.0022
[43] : train loss 0.002462, val loss drop 0.0022 to 0.0019
[56] : train loss 0.001790, val loss drop 0.0019 to 0.0015
[73] : train loss 0.002017, val loss d

In [14]:
with open(os.path.join(save_path, 'loss_info.json'), 'w') as f:
    for k in loss_per_model:
        loss_per_model[k] = np.mean(loss_per_model[k])
    f.write(json.dumps(loss_per_model))

## test

In [14]:
submission = pd.read_csv(os.path.join(root_dir, 'sample_submission.csv'))
for name in ['XY','M','V']:
    fc = classifier([128, 64, 32, 16], input_size = 512*3*5, output_size = len(name))
    conv = conv_block([16, 32, 64, 128, 256, 512], [1, 375, 5], (3, 1))
    # define model
    model = cnn_model(conv, fc)
    pred_array = []
    for i in range(1, nfold + 1):
        model.load_state_dict(torch.load(os.path.join(save_path, 'model_{}_fold{}.pt'.format(name, i))))
        model = model.cuda()

        with torch.no_grad():
            predict = model(test_f.cuda())
        pred_array.append(predict.detach().cpu().numpy())
    submission[list(name)] = np.mean(pred_array, axis = 0)

In [15]:
submission.head()

id           X           Y           M         V
0  2800 -261.139008  -39.881229  111.966736  0.434986
1  2801  316.171234 -286.514954   90.317627  0.421709
2  2802 -233.366653  128.657593   28.895294  0.357899
3  2803  160.693039  276.158539   27.653961  0.372838
4  2804 -170.325470  187.950928  133.650543  0.478190

In [16]:
submission.to_csv(os.path.join(save_path, 'submit.csv'), index = False)

In [18]:
x = torch.randn(1, 10)
y = torch.randn(1, 10)